<a href="https://colab.research.google.com/github/ayushmangupta/TF2/blob/master/VGG-TF2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
#!pip install tensorflow==2.0.0-beta1
import tensorflow as tf
import tensorflow.keras as tfk
import tensorflow.keras.layers as tfkl
import tensorflow_datasets as tfds
tfds.disable_progress_bar()
print(tf.__version__)

2.0.0-beta1


In [0]:
class VGG(tfk.Model):
  def __init__(self):
    super(VGG,self).__init__()
    
    self.common_layer = []
    self.common_layer.append(tfkl.InputLayer(input_shape=(224,224,3)))
    self.common_layer.append(tfkl.Conv2D(64,3,(1,1),padding="same",activation='relu'))
    self.common_layer.append(tfkl.Conv2D(64,3,(1,1),padding="same",activation='relu'))
    self.common_layer.append(tfkl.MaxPool2D(pool_size=(2,2),strides=(2,2)))
    self.common_layer.append(tfkl.Conv2D(128,3,(1,1),padding="same",activation='relu'))
    self.common_layer.append(tfkl.Conv2D(128,3,(1,1),padding="same",activation='relu'))
    self.common_layer.append(tfkl.MaxPool2D(pool_size=(2,2),strides=(2,2)))
    self.common_layer.append(tfkl.Conv2D(256,3,(1,1),padding="same",activation='relu'))
    self.common_layer.append(tfkl.Conv2D(256,3,(1,1),padding="same",activation='relu'))
    self.common_layer.append(tfkl.Conv2D(256,3,(1,1),padding="same",activation='relu'))
    self.common_layer.append(tfkl.Conv2D(256,3,(1,1),padding="same",activation='relu'))
    self.common_layer.append(tfkl.MaxPool2D(pool_size=(2,2),strides=(2,2)))
    self.common_layer.append(tfkl.Conv2D(512,3,(1,1),padding="same",activation='relu'))
    self.common_layer.append(tfkl.Conv2D(512,3,(1,1),padding="same",activation='relu'))
    self.common_layer.append(tfkl.Conv2D(512,3,(1,1),padding="same",activation='relu'))
    self.common_layer.append(tfkl.Conv2D(512,3,(1,1),padding="same",activation='relu'))
    self.common_layer.append(tfkl.MaxPool2D(pool_size=(2,2),strides=(2,2)))
    self.common_layer.append(tfkl.Conv2D(512,3,(1,1),padding="same",activation='relu'))
    self.common_layer.append(tfkl.Conv2D(512,3,(1,1),padding="same",activation='relu'))
    self.common_layer.append(tfkl.Conv2D(512,3,(1,1),padding="same",activation='relu'))
    self.common_layer.append(tfkl.Conv2D(512,3,(1,1),padding="same",activation='relu'))
    self.common_layer.append(tfkl.MaxPool2D(pool_size=(2,2),strides=(2,2)))
    self.common_layer.append(tfkl.Flatten())
    self.common_layer.append(tfkl.Dense(4096,activation='relu'))
    self.common_layer.append(tfkl.Dropout(.3))
    self.common_layer.append(tfkl.Dense(512,activation='relu'))
    self.common_layer.append(tfkl.Dropout(.3))
    self.common_layer.append(tfkl.Dense(64,activation='relu'))
    self.common_layer.append(tfkl.Dense(1,activation='sigmoid'))
    
  
  
  def call(self,x):
    x = tf.convert_to_tensor(tf.cast(x,dtype=tf.float32))
    #x = tf.reshape(x,(4,224,224,3))
    for layer in self.common_layer:
      x = layer(x)
      print(x.shape)
    output =x
    #print(out.shape)
    return output
    

In [0]:
model = VGG()
base_learning_rate = 0.001
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [0]:

SPLIT_WEIGHTS = (8, 1, 1)
splits = tfds.Split.TRAIN.subsplit(weighted=SPLIT_WEIGHTS)

(raw_train, raw_validation, raw_test), metadata = tfds.load(
    'cats_vs_dogs', split=list(splits),
    with_info=True, as_supervised=True)

IMG_SIZE = 224 # All images will be resized to 160x160

def format_example(image, label):
  image = tf.cast(image, tf.float32)
  image = (image/127.5) - 1
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
  return image, label

train = raw_train.map(format_example)
validation = raw_validation.map(format_example)
test = raw_test.map(format_example)
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1000

train_batches = train.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
validation_batches = validation.batch(BATCH_SIZE)
test_batches = test.batch(BATCH_SIZE)




In [0]:
model.fit(train_batches,validation_data=validation_batches,epochs=10,verbose=2)

Epoch 1/10
(None, 224, 224, 3)
(None, 224, 224, 64)
(None, 224, 224, 64)
(None, 112, 112, 64)
(None, 112, 112, 128)
(None, 112, 112, 128)
(None, 56, 56, 128)
(None, 56, 56, 256)
(None, 56, 56, 256)
(None, 56, 56, 256)
(None, 56, 56, 256)
(None, 28, 28, 256)
(None, 28, 28, 512)
(None, 28, 28, 512)
(None, 28, 28, 512)
(None, 28, 28, 512)
(None, 14, 14, 512)
(None, 14, 14, 512)
(None, 14, 14, 512)
(None, 14, 14, 512)
(None, 14, 14, 512)
(None, 7, 7, 512)
(None, 25088)
(None, 4096)
(None, 4096)
(None, 512)
(None, 512)
(None, 64)
(None, 1)


W0629 17:37:44.170618 140649479255936 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [44]:
data = tf.random.normal([10,224,224,3])
out = model(data)
print(out)

(10, 224, 224, 3)
(10, 224, 224, 64)
(10, 224, 224, 64)
(10, 112, 112, 64)
(10, 112, 112, 128)
(10, 112, 112, 128)
(10, 56, 56, 128)
(10, 56, 56, 256)
(10, 56, 56, 256)
(10, 56, 56, 256)
(10, 56, 56, 256)
(10, 28, 28, 256)
(10, 28, 28, 512)
(10, 28, 28, 512)
(10, 28, 28, 512)
(10, 28, 28, 512)
(10, 14, 14, 512)
(10, 14, 14, 512)
(10, 14, 14, 512)
(10, 14, 14, 512)
(10, 14, 14, 512)
(10, 7, 7, 512)
(10, 25088)
(10, 4096)
(10, 4096)
(10, 512)
(10, 512)
(10, 64)
(10, 1)
tf.Tensor(
[[0.49963835]
 [0.49965045]
 [0.49966988]
 [0.49968433]
 [0.49969268]
 [0.499663  ]
 [0.499668  ]
 [0.49964985]
 [0.49962276]
 [0.49960762]], shape=(10, 1), dtype=float32)


In [0]:
#model.build(input_shape=(32,224,224,3))